In [47]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import string
!pip install lxml

with open('../bin/textbook-urls.txt', 'r') as f:
    urls = [url+'/pages/preface' for url in f.read().splitlines()]


In [48]:
def getText(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    all_text = soup.get_text().lstrip('Learning Objectives').rstrip('PreviousNext')
    selection_text = re.sub('PreviousNext.+', '', all_text, flags=re.DOTALL)
    selection_text = re.sub('.+Learning Objectives', '', selection_text, flags=re.DOTALL)
    return(selection_text)
urls[:10]

['https://openstax.org/details/books/algebra-and-trigonometry/pages/preface',
 'https://openstax.org/details/books/algebra-and-trigonometry-2e/pages/preface',
 'https://openstax.org/details/books/calculus-volume-1/pages/preface',
 'https://openstax.org/details/books/calculus-volume-2/pages/preface',
 'https://openstax.org/details/books/calculus-volume-3/pages/preface',
 'https://openstax.org/details/books/college-algebra/pages/preface',
 'https://openstax.org/details/books/college-algebra-2e/pages/preface',
 'https://openstax.org/details/books/college-algebra-corequisite-support/pages/preface',
 'https://openstax.org/details/books/college-algebra-corequisite-support-2e/pages/preface',
 'https://openstax.org/details/books/elementary-algebra-2e/pages/preface']

In [64]:
def ScrapeTextbook(URL):
    textbook_split = URL.split('/')
    textbook_suffix = textbook_split[-1]
    textbook_prefix = '/'.join(textbook_split[:-1])+'/'
    print(textbook_prefix+textbook_suffix)
    
    textbook = "https://openstax.org/books/us-history/pages/"
    #req = Request(textbook + "1-introduction")
    req = Request(textbook_suffix + textbook_prefix)
    html_page = urlopen(req)
    soup = BeautifulSoup(html_page, "html.parser")

    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    
    section_urls = []
    for link in links:
        match = re.search('^[1234567890]+-', link)
        if match:
            if link.split('-')[1].isnumeric():
                section_urls.append(textbook + link)
                
    section_dict = {'section':[], 'text':[]}
    for section_url in section_urls:
        section_number = '-'.join(section_url.split('/')[-1].split('-')[:2])
        section_dict['section'].append(section_number)
        section_dict['text'].append(getText(section_url))
    
    return section_dict

In [67]:
ScrapeTextbook("https://openstax.org/books/us-history/pages/1-introduction")

https://openstax.org/books/us-history/pages/1-introduction


URLError: <urlopen error unknown url type: 1-introductionhttps>